In [1]:
import pandas as pd
import requests
import os

In [2]:
token = requests.post(
    "https://sunhat-api.onrender.com/auth/login", 
    headers={"Content-Type": "application/json"},
    json={
        "email": "victor.wagner@lmu.de",
        "password": "QFmiz5R!9TuytYL",
        }, 
    ).json()['accessToken']

In [4]:
#Upsert company (insert if not exists, update if exists. Key ist hier die ISIN). Hierfür der folgende Put-Call: 
requests.put(
    f"https://sunhat-api.onrender.com/sustainability-reports/companies/TEST",
    json={
        "name": "TEST",
        "sector": "TEST",
        "country": "Germany",
        "type": ""
    },
    headers={
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
).json()

{'isin': 'TEST', 'name': 'TEST', 'sector': 'TEST', 'country': 'DE', 'type': ''}

In [11]:
# Upload report mit folgendem Post-Call:
with open("/Users/victor/Library/CloudStorage/Dropbox/paper/esrs-study/streamlit/test.pdf", 'rb') as file:
    files = {'file': file}

    response = requests.post(
        f"https://sunhat-api.onrender.com/sustainability-reports/reports", 
        headers={"authorization": f"Bearer {token}"},
        data={
            "isin": "TEST",
            "year": "2023",
            "type": "SR"
        },
        files=files
        )

In [ ]:
response.json()

<Response [409]>

In [ ]:
"/Users/victor/Library/CloudStorage/Dropbox/paper/esrs-study/streamlit/test.pdf"

In [21]:
requests.get("https://sunhat-api.onrender.com/sustainability-reports/reports/7aa7ff34-3e28-4638-b44a-310cfde45f1e", headers={"Content-Type": "application/json"}).json()

{'id': '7aa7ff34-3e28-4638-b44a-310cfde45f1e',
 'createdAt': '2025-03-14T19:20:16.397Z',
 'link': 'https://sunhat.fra1.digitaloceanspaces.com/sustainability-reports/7aa7ff34-3e28-4638-b44a-310cfde45f1e.pdf',
 'md5': '775a1cf4c1108b6ec43684d58a2c4ab0',
 'year': 2024,
 'type': 'CSRD',
 'reportCompanyId': 'DE0005190003',
 'company': {'isin': 'DE0005190003',
  'name': 'BMW Group',
  'sector': 'TEST',
  'country': 'DE',
  'type': 'CSRD'}}

In [4]:
requests.post(
    "https://sunhat-api.onrender.com/sustainability-reports/query",
    headers={"Content-Type": "application/json"},
    json={ 
        "reportId": "8a3c0fe5-1e4c-46bb-b651-0044cd848f9f",
        "query": "What are the company's Scope 1 emission?", 
        "pageSize": 5,
        }
).json()

{'data': [],
 'pagination': {'currentPage': 1,
  'pageSize': 5,
  'totalRecords': 0,
  'totalPages': 0,
  'nextPage': None,
  'prevPage': None}}

In [35]:
requests.post(
    "https://sunhat-api.onrender.com/sustainability-reports/query",
    headers={"Content-Type": "application/json"},
    json={ 
        "reportId": "74f9a709-e1bb-418e-93fa-3daf2b8b83df",
        "query": "How is climate change ESRS E1 being handled?", 
        "pageSize": 10, 
        }
).json()

{'data': [{'reportId': '74f9a709-e1bb-418e-93fa-3daf2b8b83df',
   'page': 52,
   'x1': 419.53,
   'y1': 120.75,
   'x2': 818.39,
   'y2': 127.25,
   'score': 0.6724016075493345,
   'text': 'Sustainability Statements > Environment\nE1 | Climate Change | Page\n--- | --- | ---\nE1-1 | Transition plan for climate change mitigation | 54\nESRS 2 SBM-3-E1 | Material impacts, risks and opportunities and their interaction with strategy and business model | 55\nESRS 2 IRO-1-E1 | Description of the processes to identify and assess material climate-related impacts, risks and opportunities | 55\nE1-2 | Policies related to climate change mitigation and adaptation | 56\nE1-3 | Actions and resources in relation to climate change policies | 57\nE1-4 | Targets related to climate change mitigation and adaptation | 58\nE1-5 | Energy consumption and mix | 59\nE1-6 | Gross Scopes 1, 2, 3 and Total GHG emissions | 60',
   'link': 'https://sunhat.fra1.digitaloceanspaces.com/sustainability-reports/74f9a709-e1b

In [14]:
def get_all_reports() -> pd.DataFrame:
    """ Get all available reports from the Sunhat API """
    all_reports = []
    currentPage = 1
    pageSize = 50

    while True:
        response = requests.get(
            "https://sunhat-api.onrender.com/sustainability-reports/reports",
            headers={"Content-Type": "application/json"},
            params={"pageSize": pageSize, "page": currentPage},
        ).json()

        all_reports.extend(response.get("data"))
        
        pagination = response.get("pagination")
        if pagination.get("nextPage") is None:
            break

        currentPage += 1 

    return (
        pd.DataFrame(all_reports)
        .assign(
            companyName=lambda x: x["company"].apply(lambda y: y["name"])
            )
        .loc[:, ['id', 'companyName', 'year', 'link']]
        )

all_reports = get_all_reports()
all_reports

,id,companyName,year,link
0,035f6b32-3f58-47e8-99ce-33efd8e69255,Carlsberg,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
1,03dbd322-72ee-4f48-840b-5250fce2e8d2,Royal Schiphol Group,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
2,0a018c48-fc62-48c1-bac4-e9d840cd689c,Royal Ahold Delhaize N.V.,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
3,10368893-0376-4e48-b3f9-6a36fe2f801b,TEST,2022,https://sunhat.fra1.digitaloceanspaces.com/sus...
4,1209d194-a9f5-42d4-9e9b-7028e6565109,Demant,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
5,17df5bf2-25be-4624-b9c3-f6e7d332bb88,Covestro,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
6,18468707-5d53-477c-9159-f3615cffed54,Ringkjøbing Landbobank,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
7,1a18224d-602c-4b7e-a7cd-ea8e7194bd0c,Royal Schiphol Group,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
8,1e0c26cd-2591-4804-83d8-29b160526e87,Genmab,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
9,20ed2d1e-29ac-44c4-bc4f-eddff6b4d925,Stora Enso,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...


In [ ]:
all_reports_df = pd.DataFrame(all_reports).assign(companyName=lambda x: x["company"].apply(lambda y: y["name"])).loc[:, ['id', 'companyName', 'link']]

,id,companyName,link
0,035f6b32-3f58-47e8-99ce-33efd8e69255,Carlsberg,https://sunhat.fra1.digitaloceanspaces.com/sus...
1,03dbd322-72ee-4f48-840b-5250fce2e8d2,Royal Schiphol Group,https://sunhat.fra1.digitaloceanspaces.com/sus...
2,0a018c48-fc62-48c1-bac4-e9d840cd689c,Royal Ahold Delhaize N.V.,https://sunhat.fra1.digitaloceanspaces.com/sus...
3,1209d194-a9f5-42d4-9e9b-7028e6565109,Demant,https://sunhat.fra1.digitaloceanspaces.com/sus...
4,17df5bf2-25be-4624-b9c3-f6e7d332bb88,Covestro,https://sunhat.fra1.digitaloceanspaces.com/sus...
5,18468707-5d53-477c-9159-f3615cffed54,Ringkjøbing Landbobank,https://sunhat.fra1.digitaloceanspaces.com/sus...
6,1a18224d-602c-4b7e-a7cd-ea8e7194bd0c,Royal Schiphol Group,https://sunhat.fra1.digitaloceanspaces.com/sus...
7,1e0c26cd-2591-4804-83d8-29b160526e87,Genmab,https://sunhat.fra1.digitaloceanspaces.com/sus...
8,20ed2d1e-29ac-44c4-bc4f-eddff6b4d925,Stora Enso,https://sunhat.fra1.digitaloceanspaces.com/sus...
9,2335976a-7df1-41b9-88d0-be245de26f8d,Barclays Bank PLC,https://sunhat.fra1.digitaloceanspaces.com/sus...


## Upload

In [82]:
requests.get("https://sunhat-api.onrender.com/sustainability-reports/companies/DK0010181759").json()

{'message': 'Cannot GET /sustainability-reports/companies/DK0010181759',
 'error': 'Not Found',
 'statusCode': 404}

In [86]:
def upsert_company(company):
    """ Update or insert company data """
    try:
        response = requests.put(
            f"https://sunhat-api.onrender.com/sustainability-reports/companies/{company['isin']}",
            json={
                "name": company["company"],
                "sector": company["sector"],
                "country": company["country"],
                "type": ""
            },
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            }
        )
        response.raise_for_status()
        print(f"Successfully uploaded data")
        return True
    
    except Exception as error:
        print(f"Failed to upload data", error)
        return False
    
upsert_company({"isin": "test", "company": "test", "sector": "test", "country": "test"})

Failed to upload data 500 Server Error: Internal Server Error for url: https://sunhat-api.onrender.com/sustainability-reports/companies/test


False

In [91]:
def upload_report_file(file_path, report_data):
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return

    try:
        with open(file_path, 'rb') as file:
            files = {'file': file}
            data = {
                'isin': report_data['isin'],
                'year': report_data['year'],
                'type': report_data['type']
            }
            response = requests.post(
                "https://sunhat-api.onrender.com/sustainability-reports/reports",
                files=files,
                data=data,
                headers={"Authorization": f"Bearer {token}"}
            )
            response.raise_for_status()
        print(f"Successfully uploaded report")
        
    except Exception as error:
        print(f"Failed to upload report", error)

upload_report_file("test.pdf", {"isin": "test", "year": "0", "type": "test"})

Failed to upload report 400 Client Error: Bad Request for url: https://sunhat-api.onrender.com/sustainability-reports/reports


In [61]:
def get_documents_from_sunhat():
    """ Get all available reports from the Sunhat API """
    all_reports = []
    currentPage = 1
    pageSize = 50

    while True:
        response = requests.get(
            "https://sunhat-api.onrender.com/sustainability-reports/reports",
            headers={"Content-Type": "application/json"},
            params={"pageSize": pageSize, "page": currentPage},
        ).json()

        all_reports.extend(response.get("data"))
        
        pagination = response.get("pagination")
        if pagination.get("nextPage") is None:
            break

        currentPage += 1 

    return (
        pd.DataFrame(all_reports)
        .assign(
            company=lambda x: x["company"].apply(lambda y: y["name"])
            )
        .loc[:, ['id', 'company', 'link']]
        )

docs_sunhat = get_documents_from_sunhat()

In [ ]:
def get_documents_from_sheet():
    return (
        # Read data from Google Sheet
        pd.read_csv("https://docs.google.com/spreadsheets/d/1Nlyf8Yz_9Fst8rEmQc2IMc-DWLF1fpmBTB7n4FlZwxs/export?format=csv&gid=0", skiprows=2)
        .query("verified == 'yes'")
        .rename(columns={
            'SASB industry \n(SICS® Industries)': "industry",
            })
        .merge(
            # Merge industry-sector lookup from separate sheet
            pd.read_csv(
                "https://docs.google.com/spreadsheets/d/1Nlyf8Yz_9Fst8rEmQc2IMc-DWLF1fpmBTB7n4FlZwxs/export?format=csv&gid=218767986#gid=218767986"
                ).rename(columns={
                    "SICS® Industries": "industry",
                    "SICS® Sector": "sector"
                    }
                ), on="industry", how="left"
            )
        .assign(
            company = lambda x: x["company"].str.strip(),
        )
        .loc[:, ['company', 'link']]
    )

docs_sheet = get_documents_from_sheet()

In [65]:
(
    pd.merge(
        docs_sheet, docs_sunhat, 
        on="company", 
        how="outer", 
        indicator=True, 
        suffixes=("_sheet", "_sunhat")
    )
    .query("_merge == 'left_only'")
)

,company,link_sheet,id,link_sunhat,_merge
0,AB InBev,https://cdn.builder.io/o/assets%2F2e5c7fb02019...,NaN,NaN,left_only
1,ALK,https://ir.alk.net/static-files/b6b65921-c84e-...,NaN,NaN,left_only
3,ASML Holding NV,https://ourbrand.asml.com/m/79d325b168e0fd7e/o...,NaN,NaN,left_only
4,Acerinox,https://www.acerinox.com/export/sites/acerinox...,NaN,NaN,left_only
5,Aena,https://www.aena.es/sites/Satellite?blobcol=ur...,NaN,NaN,left_only
...,...,...,...,...,...
105,Trifork Group,https://investor.trifork.com/wp-content/upload...,NaN,NaN,left_only
107,UCB,https://djmyn0vvwsj0h.cloudfront.net/UCB_IAR_2...,NaN,NaN,left_only
110,WithSecure,https://www.withsecure.com/content/dam/with-se...,NaN,NaN,left_only
112,Wärtsilä,https://mb.cision.com/Main/15003/4107848/32729...,NaN,NaN,left_only


In [9]:
get_all_reports()

,id,companyName,year,link
0,035f6b32-3f58-47e8-99ce-33efd8e69255,Carlsberg,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
1,03dbd322-72ee-4f48-840b-5250fce2e8d2,Royal Schiphol Group,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
2,0a018c48-fc62-48c1-bac4-e9d840cd689c,Royal Ahold Delhaize N.V.,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
3,1209d194-a9f5-42d4-9e9b-7028e6565109,Demant,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
4,17df5bf2-25be-4624-b9c3-f6e7d332bb88,Covestro,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
5,18468707-5d53-477c-9159-f3615cffed54,Ringkjøbing Landbobank,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
6,1a18224d-602c-4b7e-a7cd-ea8e7194bd0c,Royal Schiphol Group,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
7,1e0c26cd-2591-4804-83d8-29b160526e87,Genmab,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
8,20ed2d1e-29ac-44c4-bc4f-eddff6b4d925,Stora Enso,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...
9,2335976a-7df1-41b9-88d0-be245de26f8d,Barclays Bank PLC,2024,https://sunhat.fra1.digitaloceanspaces.com/sus...


In [15]:
requests.post(
        "https://sunhat-api.onrender.com/sustainability-reports/query",
        headers={"Content-Type": "application/json"},
        json={ 
            "reportId": "ec2dce12-ab23-4f00-adec-38a666ee0e6f",
            "query": "Who are the authors?", 
            "pageSize": 1, 
            }
    ).json()

{'data': [{'reportId': 'ec2dce12-ab23-4f00-adec-38a666ee0e6f',
   'page': 9,
   'x1': 38.98,
   'y1': 130.43,
   'x2': 319.88,
   'y2': 154.89,
   'score': 0.37579184770584106,
   'text': 'Appendix: Finding topics in sustainability reports\nReach out to Victor Wagner (victor.wagner@lmu.de) or visit our homepage!',
   'link': 'https://sunhat.fra1.digitaloceanspaces.com/sustainability-reports/ec2dce12-ab23-4f00-adec-38a666ee0e6f.pdf'}],
 'pagination': {'currentPage': 1,
  'pageSize': 1,
  'totalRecords': 53,
  'totalPages': 53,
  'nextPage': 2,
  'prevPage': None}}

In [18]:
requests.get("https://sunhat-api.onrender.com/sustainability-reports/reports").json()

{'data': [{'id': '035f6b32-3f58-47e8-99ce-33efd8e69255',
   'createdAt': '2025-02-27T14:30:29.586Z',
   'link': 'https://sunhat.fra1.digitaloceanspaces.com/sustainability-reports/035f6b32-3f58-47e8-99ce-33efd8e69255.pdf',
   'md5': 'a9abbf836f7ace06effb0dd77e056e95',
   'year': 2024,
   'type': 'CSRD',
   'reportCompanyId': 'DK0010181759',
   'company': {'isin': 'DK0010181759',
    'name': 'Carlsberg',
    'sector': 'Alcoholic Beverages',
    'country': 'DK',
    'type': ''}},
  {'id': '03dbd322-72ee-4f48-840b-5250fce2e8d2',
   'createdAt': '2025-02-27T14:31:02.476Z',
   'link': 'https://sunhat.fra1.digitaloceanspaces.com/sustainability-reports/03dbd322-72ee-4f48-840b-5250fce2e8d2.pdf',
   'md5': 'a464da5b2cdd61313024ef585b83a942',
   'year': 2024,
   'type': 'CSRD',
   'reportCompanyId': 'n.a.',
   'company': {'isin': 'n.a.',
    'name': 'Royal Schiphol Group',
    'sector': 'Air Freight & Logistics',
    'country': 'NL',
    'type': ''}},
  {'id': '0a018c48-fc62-48c1-bac4-e9d840cd689

In [ ]:
token = requests.post(
    "https://sunhat-api.onrender.com/auth/login", 
    headers={"Content-Type": "application/json"},
    json={
        "email": "victor.wagner@lmu.de",
        "password": "QFmiz5R!9TuytYL",
        }, 
    ).json()['accessToken']

In [ ]:
response = requests.post(
    f"https://sunhat-api.onrender.com/sustainability-reports/companies/DE0005773303", 
    headers={"authorization": f"Bearer {token}"},
    data={
        "isin": "DE0005773303",
        "year": "2022",
        "type": "SR"
    },
    files={ "file": "open('/Users/victor/Library/CloudStorage/Dropbox/paper/esrs-study/streamlit/test.pdf', 'rb')" }, 
    )

response.json()

{'message': 'Unexpected token \'-\', "--dc1dc9d2"... is not valid JSON',
 'error': 'Bad Request',
 'statusCode': 400}

In [21]:
pd.read_json("/Users/victor/Library/CloudStorage/Dropbox/paper/esrs-study/reports/early-analyses-csrd-pdfelements/DK0060336014_2024_AR.json").drop("path", axis=1).to_excel("novonesis-pdf_elements.xlsx")

In [10]:
def get_all_reports() -> pd.DataFrame:
    """ Get all available reports from the Sunhat API """
    all_reports = []
    currentPage = 1
    pageSize = 50

    print("Fetching reports from Sunhat API...")

    while True:
        response = requests.get(
            "https://sunhat-api.onrender.com/sustainability-reports/reports",
            headers={"Content-Type": "application/json"},
            params={"pageSize": pageSize, "page": currentPage},
        ).json()

        all_reports.extend(response.get("data"))
        
        pagination = response.get("pagination")
        if pagination.get("nextPage") is None:
            break

        currentPage += 1 

    return (
        pd.DataFrame(all_reports)
        .assign(
            companyName=lambda x: x["company"].apply(lambda y: y["name"])
            )
        .loc[:, ['id', 'companyName', 'type', 'link']]
        )

reports = get_all_reports()

Fetching reports from Sunhat API...


In [11]:
reports.query('type != "AR"')

,id,companyName,type,link
32,2f539b6f-3aa5-496b-8c2b-0331bc194a09,Demant,CSRD,https://sunhat.fra1.digitaloceanspaces.com/sus...
79,7d93906f-d996-4b43-94db-8900092e52f0,K+S,CSRD,https://sunhat.fra1.digitaloceanspaces.com/sus...
127,e2c2124c-fd33-4a29-8d67-b26e92d09816,BMW Group,CSRD,https://sunhat.fra1.digitaloceanspaces.com/sus...
